__ this is the current working version with query __

In [1]:
import pandas as pd
import plotly.graph_objects as go
import time
import requests
import yaml
from datetime import datetime
from sqlalchemy import create_engine
import urllib
import dash
from dash import html, dcc, Input, Output

# --- New Credentials (hard-coded) ---
server = "VM-LESSQL04"      # Replace with your new server address
database = "LOPR"       # Replace with your new database name
# username = ""            # Replace with your new username
# password = ""            # Replace with your new password

# Optional: Print out the credentials for debugging purposes
print(f"Server: {server}")
print(f"Database: {database}")
# print(f"Username: {username}")

# --- Create the ODBC connection string and URL-encode it ---
params = urllib.parse.quote_plus(
    "DRIVER={SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    "Trusted_Connection=yes;"
)

# --- Create SQLAlchemy engine using the pyodbc driver ---
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# SQL query (unchanged)
sql_query = """
WITH Temp_sv302SigleReturn AS (
    SELECT
        sv302.[Service_Call_ID],
        sv302.[System],
        sv302.[Task_Code],
        TRY_CAST(sv302.[Task_User_Define13] AS INT) AS [Task_User_Define13],
        sv302.[Task_User_Define5],
        CAST(sv302.[CREATDDT] AS DATE) AS [CREATDDT],
        CAST(sv302.[Task_User_Define9] AS DATE) AS [Task_User_Define9],
        CAST(sv302.[Task_User_Define10] AS DATE) AS [Task_User_Define10],
        ROW_NUMBER() OVER (
            PARTITION BY sv302.[Service_Call_ID]
            ORDER BY sv302.[DEX_ROW_ID] DESC
        ) AS [ROW_NUMBER]
    FROM [VM-LESSQL04].[LOPR].[dbo].[SV00302] sv302
    WHERE 
        sv302.[Major] = 'TRAP'
        AND sv302.[Equipment_ID] NOT LIKE 'FEES'
        AND sv302.[System] IS NOT NULL
        AND sv302.[Task_Code] NOT LIKE 'QT-TRCH'
        AND (sv302.[Service_Call_ID] LIKE '25%' OR sv302.[Service_Call_ID] LIKE '24%')
),
ExistingServices AS (
    SELECT 
        LEFT(svc.[Region], 3) AS [SITE],
        RIGHT('000000' + CAST(svc.[Customer_Number] AS VARCHAR(6)), 6)
            + RIGHT('00000' + CAST(svc.[Address_Code] AS VARCHAR(5)), 5) AS [ACCOUNT],
        ts.[System] AS [LOB],
        CASE 
            WHEN svc.[Complete] = 0 THEN 'not routed'
            WHEN svc.[Complete] = 1 THEN 'routed'
            WHEN svc.[Complete] = 2 THEN 'serviced'
            WHEN svc.[Complete] IN (3, 4, 9) THEN 'complete'
            ELSE 'unknown'
        END AS [CompletionStatus],
        ts.[Task_User_Define5] AS [FREQ],
        ts.[Task_User_Define10] AS [NSD],
        ts.[Task_User_Define13] AS [GALLONS],
        svc.[Customer_Number],
        svc.[Address_Code]
    /*svc*/
    FROM [VM-TMWDB].[Services].[dbo].[Services] svc
    /*ts*/
    INNER JOIN Temp_sv302SigleReturn ts
        ON svc.[Service_Call_ID] = ts.[Service_Call_ID]
        AND ts.[ROW_NUMBER] = 1
    /*sv500*/
    INNER JOIN [VM-LESSQL04].[LOPR].[dbo].[SV00500] sv500
        ON svc.[Customer_Number] = sv500.[CUSTNMBR]
        AND svc.[Address_Code] = sv500.[ADRSCODE]
    /*svcData*/
    LEFT JOIN [VM-TMWDB].[Services].[dbo].[Services_Data] svcData
        ON svc.[Service_Call_ID] = svcData.[Service_Call_ID]
        AND svc.[Appointment] = svcData.[Appointment]
    /*sv301*/
    LEFT JOIN [VM-LESSQL04].[LOPR].[dbo].[SV00301] sv301
        ON svc.[Service_Call_ID] = LTRIM(RTRIM(sv301.[Service_Call_ID]))
        AND svc.[Appointment] = CAST(LTRIM(RTRIM(sv301.[Appointment])) AS INT)
    WHERE 
        (svc.[Complete] = 0 OR svc.[Complete] = 1)
        AND (svcData.[ApprovedBy] IS NULL OR svcData.[ApprovedBy] <> 'WS.Deleted')
        AND sv500.[HOLD] = 0
        AND sv301.[Appointment_Status] NOT LIKE 'ON HOLD'
        AND (ts.[System] LIKE 'GRTR' OR ts.[System] LIKE 'OWS' OR ts.[System] LIKE 'GRIT' OR ts.[System] LIKE 'UCO')
),
ProjectionServices AS (
    SELECT DISTINCT
        LEFT(sv200.[Divisions], 3) AS [SITE],
        RIGHT('000000' + CAST(sv582.[CUSTNMBR] AS VARCHAR(6)), 6)
            + RIGHT('00000' + CAST(sv582.[ADRSCODE] AS VARCHAR(5)), 5) AS [ACCOUNT],
        sv582.[System] AS [LOB],
        'projection' AS [CompletionStatus],
        sv582.[Task_User_Define5] AS [FREQ],
        CAST(sv582.[Task_User_Define10] AS DATE) AS [NSD],
        sv582.[Task_User_Define13] AS [GALLONS],
        sv582.[CUSTNMBR] AS [Customer_Number],
        sv582.[ADRSCODE] AS [Address_Code]
    /*sv582*/
    FROM [VM-LESSQL04].[LOPR].[dbo].[SV00582] sv582
    /*sv560*/
    INNER JOIN [VM-LESSQL04].[LOPR].[dbo].[SV00560] sv560
        ON sv560.Task_Code = sv582.Task_Code
        AND sv560.Major = 'TRAP'
    /*sv200*/
    INNER JOIN [VM-LESSQL04].[LOPR].[dbo].[SV00200] sv200
        ON sv200.[CUSTNMBR] = sv582.[CUSTNMBR]
        AND sv200.[ADRSCODE] = sv582.[ADRSCODE]
    /*sv500*/
    INNER JOIN [VM-LESSQL04].[LOPR].[dbo].[SV00500] sv500
        ON sv582.[CUSTNMBR] = sv500.[CUSTNMBR]
        AND sv582.[ADRSCODE] = sv500.[ADRSCODE]
    WHERE 
        sv582.[Task_User_Define5] <> 0
        AND sv200.[WSReserved_CB1] = 0
        AND sv500.[HOLD] = 0
        AND sv560.[Major] = 'TRAP'
        AND sv582.[Task_User_Define10] > '2024-01-01'
        AND sv582.[Task_User_Define9] > '2024-01-01'
        AND (sv582.[System] LIKE 'GRTR' OR sv582.[System] LIKE 'OWS' OR sv582.[System] LIKE 'GRIT' OR sv582.[System] LIKE 'UCO')
        AND sv582.[System] NOT LIKE 'GREASEFEES'
        AND sv582.[System] IS NOT NULL
        AND sv582.[System] NOT LIKE '               '
        AND sv582.[Task_Code] NOT LIKE 'QT-TRCH'
        AND NOT EXISTS (
            SELECT 1 
            FROM ExistingServices es 
            WHERE es.[Customer_Number] = sv582.[CUSTNMBR]
            AND es.[Address_Code] = sv582.[ADRSCODE]
        )
)
SELECT 
    SITE, 
    ACCOUNT, 
    LOB, 
    CompletionStatus, 
    FREQ, 
    NSD, 
    GALLONS, 
    Customer_Number, 
    Address_Code
FROM ExistingServices
UNION ALL
SELECT 
    SITE, 
    ACCOUNT, 
    LOB, 
    CompletionStatus, 
    FREQ, 
    NSD, 
    GALLONS, 
    Customer_Number, 
    Address_Code
FROM ProjectionServices;

"""

# Fetch data using the SQLAlchemy engine
df = pd.read_sql(sql_query, engine)
print(f"Number of rows returned from SQL query: {len(df)}")

# Convert FREQ, NSD, and GALLONS to appropriate data types
df['FREQ'] = pd.to_numeric(df['FREQ'], errors='coerce')
df['NSD'] = pd.to_datetime(df['NSD'], errors='coerce')
df['GALLONS'] = pd.to_numeric(df['GALLONS'], errors='coerce')

# Extract unique LOB values from the query
lob_list = df['LOB'].unique().tolist()

# Get today's date and ISO week number
today = pd.Timestamp('today').normalize()
current_week = today.isocalendar()[1]
current_year = today.year

# Global parameter for projection
predict = 26
weekCodes = list(range(1, predict + 1))

# Calculate base Sunday date: find the upcoming Sunday
# days_until_sunday = (6 - today.weekday()) % 7
# base_sunday = today + pd.Timedelta(days=days_until_sunday)

days_since_sunday = (today.weekday() + 1) % 7
base_sunday = today - pd.Timedelta(days=days_since_sunday)

# Create list of Sunday dates for each forecast week
sunday_dates = [(base_sunday + pd.Timedelta(weeks=i)).strftime("%Y-%m-%d") for i in range(predict)]

# Function to create the source projection using a fixed count
def create_customer_projection(start_week_offset, frequency, count_value):
    projection = pd.Series(0, index=weekCodes)
    for week in range(start_week_offset, predict + 1, frequency):
        projection[week] = count_value
    return projection

# Generate projections for each customer (using a count for SC)
source_projections = []
included_rows = 0
excluded_rows = 0
for idx, row in df.iterrows():
    try:
        # Existing check for NSD in the past
        if row['NSD'] < today and row['CompletionStatus'] in ['not routed', 'routed'] and row['CompletionStatus'] != 'projection':
            start_week_offset = 1
        else:
            # Calculate the NSD's ISO week; adjust if NSD is next year.
            nsd_week = row['NSD'].isocalendar()[1]
            nsd_year = row['NSD'].year
            if nsd_year > current_year:
                nsd_week += 52
            week_difference = nsd_week - current_week

            frequency = int(row['FREQ'])
            # Determine skew based on FREQ using strict inequalities
            if frequency < 10:
                skew = 1
            elif 10 < frequency < 17:
                skew = 2
            elif frequency > 17:
                skew = 4
            else:
                # Handle edge cases: for FREQ exactly 10 or 17, choose a default:
                # For example, assign skew=1 if FREQ==10 and skew=2 if FREQ==17.
                skew = 1 if frequency == 10 else 2

            # Adjust start_week_offset by subtracting the skew
            start_week_offset = max(1, week_difference + 1 - skew)

            if start_week_offset < 1 or start_week_offset > predict:
                excluded_rows += 1
                continue

        service_count = 1  
        projection = create_customer_projection(start_week_offset, frequency, service_count)
        source_projections.append((row['SITE'], row['LOB'], projection))
        included_rows += 1

    except Exception:
        excluded_rows += 1
        continue


print(f"Total rows included: {included_rows}")
print(f"Total rows excluded: {excluded_rows}")

# Convert projections to DataFrame, including LOB
projection_df = pd.DataFrame([
    {'SITE': proj[0], 'LOB': proj[1], **proj[2].to_dict()} for proj in source_projections
])

# Group by SITE and LOB and sum the projections
grouped_source = projection_df.groupby(['SITE', 'LOB'])[weekCodes].sum().reset_index()

# Create the Site-to-Region mapping DataFrame
site_region_df = pd.DataFrame({
    'SITE': ['ATL', 'ATB', 'AUB', 'AUS', 'BAL', 'BIR', 'CHR', 'CHI', 'CLE', 'DAL', 'DEN', 'HOU',
             'JAX', 'KAN', 'KNO', 'LSV', 'LIT', 'LUF', 'MEM', 'MIA', 'MIL', 'MIN', 'MOB',
             'NAS', 'NOR', 'NCA', 'OKC', 'PHI', 'PHX', 'PIT', 'POR', 'RGV', 'SAN', 'SND',
             'SEA', 'SHR', 'SCA', 'STL', 'TAM', 'WSM'],
    'Region': ['East', 'North', 'North', 'West', 'North', 'East', 'East', 'North', 'North', 'West',
               'West', 'West', 'East', 'North', 'East', 'West', 'East', 'West', 'East', 'East',
               'North', 'North', 'East', 'East', 'East', 'West', 'West', 'East', 'West', 'North',
               'West', 'West', 'West', 'West', 'West', 'East', 'West', 'North', 'East', 'East']
})

# Merge grouped_source with site_region_df on SITE
grouped_source = grouped_source.merge(site_region_df, on='SITE', how='left')

# Get lists for dropdowns
site_list = grouped_source['SITE'].unique().tolist()
region_list = site_region_df['Region'].unique().tolist()

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout with added LOB dropdown
app.layout = html.Div([
    html.Div([
        html.Label('Select Regions:', style={'color': '#00ff41'}),
        dcc.Dropdown(
            id='region-dropdown',
            options=[{'label': region, 'value': region} for region in sorted(region_list)],
            value=[],
            multi=True,
            placeholder="Select Regions",
            style={'backgroundColor': 'rgb(30, 30, 30)', 'color': 'green', 'width': '100%'}
        )
    ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    
    html.Div([
        html.Label('Select LOB:', style={'color': '#00ff41'}),
        dcc.Dropdown(
            id='lob-dropdown',
            options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': lob, 'value': lob} for lob in sorted(lob_list)],
            value=[],
            multi=True,
            placeholder="Select LOB",
            style={'backgroundColor': 'rgb(30, 30, 30)', 'color': 'green', 'width': '100%'}
        )
    ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '2%', 'verticalAlign': 'top'}),

    html.Div([
        html.Label('Select Sites:', style={'color': '#00ff41'}),
        dcc.Dropdown(
            id='site-dropdown',
            options=[{'label': site, 'value': site} for site in sorted(site_list)],
            value=site_list[:5],
            multi=True,
            placeholder="Select Sites",
            style={'backgroundColor': 'rgb(30, 30, 30)', 'color': 'green', 'width': '100%'}
        )
    ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '2%', 'verticalAlign': 'top'}),

    dcc.Graph(id='graph', style={'height': '550px'}) # Adjust height as needed
], style={'backgroundColor': 'rgb(30, 30, 30)', 'padding': '10px'})

# Figure creation function using filtered grouped_source data with Sunday dates on the x-axis
def create_figure(filtered_data):
    fig = go.Figure()
    # Exclude week 1 from the visualization by slicing weekCodes and sunday_dates
    plot_weekCodes = weekCodes[1:]  # assuming weekCodes is something like [1, 2, ..., predict]
    plot_sunday_dates = sunday_dates[1:]
    
    for _, row in filtered_data.iterrows():
        site = row['SITE']
        # Convert the projection row to a list, then slice off the first element
        y_values = [row[week] for week in weekCodes][1:]
        fig.add_trace(go.Scatter(
            x=plot_sunday_dates,  # use the modified sunday_dates excluding the current week
            y=y_values,
            mode='lines+markers',
            name=site,
            line=dict(dash='solid', shape='spline'),
            visible=True
        ))
    fig.update_layout(
        xaxis_title="Sunday Date",
        yaxis_title="Service Count",
        xaxis=dict(
            tickmode='array',
            tickvals=plot_sunday_dates,
            ticktext=plot_sunday_dates,
            gridcolor='gray',
            zerolinecolor='gray',
            gridwidth=0.5,
            type='category'
        ),
        yaxis=dict(
            gridcolor='gray',
            zerolinecolor='gray',
            gridwidth=0.5,
        ),
        paper_bgcolor='rgb(30, 30, 30)',
        plot_bgcolor='rgb(40, 40, 40)',
        font=dict(color='#00ff41'),
        hovermode='x',
        legend_title="Site",
        height=900,
        margin=dict(l=40, r=20, t=20, b=40)
    )
    return fig

# Callback to update the graph based on selected Sites, Regions, and LOB
@app.callback(
    Output('graph', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('region-dropdown', 'value'),
     Input('lob-dropdown', 'value')]
)
def update_graph(selected_sites, selected_regions, selected_lobs):
    filtered = grouped_source.copy()
    if selected_regions:
        sites_in_regions = site_region_df[site_region_df['Region'].isin(selected_regions)]['SITE']
        filtered = filtered[filtered['SITE'].isin(sites_in_regions)]
    
    # Handle LOB filtering:
    if selected_lobs:
        if 'ALL' not in selected_lobs:
            filtered = filtered[filtered['LOB'].isin(selected_lobs)]
        else:
            # When "ALL" is selected, aggregate across LOBs for each SITE
            # Group by SITE and sum all the weekly count columns
            filtered = filtered.groupby('SITE', as_index=False)[weekCodes].sum()
            # Optionally add a dummy LOB column to label the aggregated data
            filtered['LOB'] = 'ALL'
    
    if selected_sites:
        filtered = filtered[filtered['SITE'].isin(selected_sites)]
    
    if filtered.empty:
        return go.Figure(layout={
            'paper_bgcolor': 'rgb(30, 30, 30)',
            'plot_bgcolor': 'rgb(40, 40, 40)',
            'font': {'color': 'green'},
            'xaxis': {'visible': False},
            'yaxis': {'visible': False},
            'annotations': [{
                'text': 'Please select at least one site, region, or LOB.',
                'xref': 'paper',
                'yref': 'paper',
                'showarrow': False,
                'font': {'size': 20, 'color': 'green'}
            }]
        })
    else:
        return create_figure(filtered)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

Server: VM-LESSQL04
Database: LOPR
Number of rows returned from SQL query: 85182
Total rows included: 80402
Total rows excluded: 4780


__ output HTML files __

In [18]:
import os

# Set the output directory for exported HTML files
output_directory = r'C:\Users\aaron.eades\OneDrive - Liquid Environmental Solutions\Documents\Programing\Python\Automated Density\exports'

# Ensure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Get the unique Regions and LOBs from the data
unique_regions = grouped_source['Region'].dropna().unique().tolist()
unique_lobs = grouped_source['LOB'].dropna().unique().tolist()

# Loop through each combination of Region and LOB
for region in sorted(unique_regions):
    for lob in sorted(unique_lobs):
        # Filter the data for the current combination
        filtered_data = grouped_source[(grouped_source['Region'] == region) &
                                        (grouped_source['LOB'] == lob)]
        if filtered_data.empty:
            print(f"No data for Region '{region}' and LOB '{lob}'. Skipping.")
            continue
        
        # Create the figure using your existing function
        fig = create_figure(filtered_data)
        
        # Construct an output file name; replace spaces with underscores if necessary
        safe_region = region.replace(" ", "_")
        safe_lob = lob.replace(" ", "_")
        output_file_name = f"projection_figure.{safe_region}.{safe_lob}.html"
        output_file_path = os.path.join(output_directory, output_file_name)
        
        # Save the figure as an HTML file
        fig.write_html(output_file_path)
        print(f"Saved file for Region '{region}' and LOB '{lob}': {output_file_path}")


Saved file for Region 'East' and LOB 'GRIT           ': C:\Users\aaron.eades\OneDrive - Liquid Environmental Solutions\Documents\Programing\Python\Automated Density\exports\projection_figure.East.GRIT___________.html
Saved file for Region 'East' and LOB 'GRTR           ': C:\Users\aaron.eades\OneDrive - Liquid Environmental Solutions\Documents\Programing\Python\Automated Density\exports\projection_figure.East.GRTR___________.html
Saved file for Region 'East' and LOB 'OWS            ': C:\Users\aaron.eades\OneDrive - Liquid Environmental Solutions\Documents\Programing\Python\Automated Density\exports\projection_figure.East.OWS____________.html
Saved file for Region 'East' and LOB 'UCO            ': C:\Users\aaron.eades\OneDrive - Liquid Environmental Solutions\Documents\Programing\Python\Automated Density\exports\projection_figure.East.UCO____________.html
Saved file for Region 'North' and LOB 'GRIT           ': C:\Users\aaron.eades\OneDrive - Liquid Environmental Solutions\Documents\Pr

In [23]:
import plotly.io as pio

# Assuming 'fig' is your Plotly figure you want to export
output_file = r'C:\Users\aaron.eades\OneDrive - Liquid Environmental Solutions\Documents\Programing\Python\Automated Density\exports\static_plotly_figure.html'
pio.write_html(fig, file=output_file, auto_open=True)
print("Static Plotly figure exported to:", output_file)

Static Plotly figure exported to: C:\Users\aaron.eades\OneDrive - Liquid Environmental Solutions\Documents\Programing\Python\Automated Density\exports\static_plotly_figure.html
